# HDB Resale Price Prediction

## Data Blending

### Import Libraries

In [2]:
# General
import re
import numpy as np
import pandas as pd
from datetime import datetime
from tabulate import tabulate
from math import radians
from sklearn.metrics.pairwise import haversine_distances

# Warnings
import warnings
warnings.filterwarnings('ignore')

### Import HDB Dataset

In [3]:
# Importing Data
data_hdb_raw = pd.read_csv('../dataset/hdb_latest_raw.csv')
data_hdb_raw

,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,block,remaining_lease,address,full_address,lat,long,nearest_mrt,nearest_distance_to_mrt
0,0,ANG MO KIO,3 ROOM,Improved,69.0,ANG MO KIO AVE 4,147000.0,2000-01,1986,07 TO 09,170,85,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
1,1,ANG MO KIO,3 ROOM,Improved,60.0,ANG MO KIO AVE 4,130000.0,2000-02,1986,07 TO 09,170,85,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
2,2,ANG MO KIO,2 ROOM,Improved,45.0,ANG MO KIO AVE 4,105000.0,2000-03,1986,04 TO 06,170,85,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
3,3,ANG MO KIO,2 ROOM,Improved,45.0,ANG MO KIO AVE 4,125000.0,2000-05,1986,04 TO 06,170,85,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
4,4,ANG MO KIO,2 ROOM,Improved,45.0,ANG MO KIO AVE 4,117000.0,2000-06,1986,07 TO 09,170,85,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881919,279698,WOODLANDS,5 ROOM,IMPROVED,126.0,WOODLANDS ST 83,316000.0,1999-12,1996,10 TO 12,863,96,863 WOODLANDS ST 83,863 WOODLANDS STREET 83 MY WORLD PRESCHOOL LTD...,1.440397,103.795020,admiralty,0.664
881920,279699,WOODLANDS,5 ROOM,IMPROVED,129.0,WOODLANDS ST 83,318000.0,1999-12,1996,04 TO 06,863,96,863 WOODLANDS ST 83,863 WOODLANDS STREET 83 MY WORLD PRESCHOOL LTD...,1.440397,103.795020,admiralty,0.664
881921,279700,WOODLANDS,5 ROOM,IMPROVED,124.0,WOODLANDS ST 83,311000.0,1999-12,1996,04 TO 06,863,96,863 WOODLANDS ST 83,863 WOODLANDS STREET 83 MY WORLD PRESCHOOL LTD...,1.440397,103.795020,admiralty,0.664
881922,279701,WOODLANDS,5 ROOM,IMPROVED,127.0,WOODLANDS ST 83,355000.0,1999-12,1996,01 TO 03,864,96,864 WOODLANDS ST 83,864 WOODLANDS STREET 83 SINGAPORE 730864,1.440847,103.794991,admiralty,0.668


### Truncate Dataset

In [4]:
# Reduce to Last 15 Years i.e. 2009 onwards
data_hdb_last15 = data_hdb_raw
data_hdb_last15['month'] = pd.to_datetime(data_hdb_last15['month'])
data_hdb_last15 = data_hdb_last15[data_hdb_last15['month'].dt.year >= 2009]
data_hdb_last15

,Unnamed: 0,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,lease_commence_date,storey_range,block,remaining_lease,address,full_address,lat,long,nearest_mrt,nearest_distance_to_mrt
77,77,ANG MO KIO,3 ROOM,Improved,61.0,ANG MO KIO AVE 4,200000.0,2009-03-01,1986,04 TO 06,170,76,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
78,78,ANG MO KIO,3 ROOM,Improved,60.0,ANG MO KIO AVE 4,200000.0,2009-04-01,1986,04 TO 06,170,76,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
79,79,ANG MO KIO,3 ROOM,Improved,69.0,ANG MO KIO AVE 4,224000.0,2009-06-01,1986,04 TO 06,170,76,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
80,80,ANG MO KIO,2 ROOM,Improved,45.0,ANG MO KIO AVE 4,170000.0,2009-07-01,1986,07 TO 09,170,76,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
81,81,ANG MO KIO,3 ROOM,Improved,61.0,ANG MO KIO AVE 4,193000.0,2009-07-01,1986,04 TO 06,170,76,170 ANG MO KIO AVE 4,170 ANG MO KIO AVENUE 4 KEBUN BARU LINK 1 SING...,1.374001,103.836432,mayflower,0.283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602216,602216,YISHUN,2 ROOM,Model A,38.0,YISHUN AVE 4,335000.0,2023-01-01,2018,07 TO 09,675A,94 years 11 months,675A YISHUN AVE 4,675A YISHUN AVENUE 4 FERN GROVE @ YISHUN SINGA...,1.419458,103.843092,khatib,1.147
602217,602217,YISHUN,4 ROOM,Model A,93.0,YISHUN AVE 4,540000.0,2023-01-01,2018,04 TO 06,675A,94 years 11 months,675A YISHUN AVE 4,675A YISHUN AVENUE 4 FERN GROVE @ YISHUN SINGA...,1.419458,103.843092,khatib,1.147
602218,602218,YISHUN,3 ROOM,Model A,68.0,YISHUN AVE 4,450000.0,2023-01-01,2018,10 TO 12,673C,94 years 11 months,673C YISHUN AVE 4,673C YISHUN AVENUE 4 FERN GROVE @ YISHUN SINGA...,1.421049,103.842207,khatib,1.104
602219,602219,YISHUN,5 ROOM,Improved,112.0,YISHUN AVE 4,690000.0,2023-01-01,2018,07 TO 09,673C,94 years 11 months,673C YISHUN AVE 4,673C YISHUN AVENUE 4 FERN GROVE @ YISHUN SINGA...,1.421049,103.842207,khatib,1.104


In [5]:
# Re-index the dataframe
data_hdb_last15 = data_hdb_last15.reset_index(drop=True)
data_hdb_last15.index += 1

In [6]:
# Understanding Data I
data_hdb_last15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325970 entries, 1 to 325970
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Unnamed: 0               325970 non-null  int64         
 1   town                     325970 non-null  object        
 2   flat_type                325970 non-null  object        
 3   flat_model               325970 non-null  object        
 4   floor_area_sqm           325970 non-null  float64       
 5   street_name              325970 non-null  object        
 6   resale_price             325970 non-null  float64       
 7   month                    325970 non-null  datetime64[ns]
 8   lease_commence_date      325970 non-null  int64         
 9   storey_range             325970 non-null  object        
 10  block                    325970 non-null  object        
 11  remaining_lease          325970 non-null  object        
 12  address         

### Data Wrangling

In [7]:
# Drop Unnamed First (Index) Column
data_hdb_last15 = data_hdb_last15.drop(data_hdb_last15.columns[0], axis = 1)

In [8]:
# Merge flat model with different names that refer to the same type
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('MODEL A','Model A')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('IMPROVED','Improved')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('NEW GENERATION','New Generation')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('PREMIUM APARTMENT', 'Premium Apartment')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('SIMPLIFIED','Simplified')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('APARTMENT','Apartment')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('STANDARD','Standard')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('MAISONETTE','Maisonette')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('2-ROOM', '2-room')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('IMPROVED-MAISONETTE', 'Improved-Maisonette')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('MODEL A-MAISONETTE', 'Model A-Maisonette')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('MULTI GENERATION', 'Multi Generation')
data_hdb_last15['flat_model'] = data_hdb_last15['flat_model'].str.replace('TERRACE','Terrace')

In [9]:
# Find out the different types of flat models for each flat type
grouped = data_hdb_last15.groupby('flat_type')['flat_model'].unique()
results = []

for flat_type, flat_models in grouped.items():
    results.append([flat_type, ', '.join(map(str, flat_models))])

print(tabulate(results, headers=["Flat Type", "Flat Models"], tablefmt="pretty"))

+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|    Flat Type     |                                                                     Flat Models                                                                     |
+------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------+
|      1 ROOM      |                                                                      Improved                                                                       |
|      2 ROOM      |                                            Improved, Standard, 2-room, Model A, Premium Apartment, DBSS                                             |
|      3 ROOM      |                              Improved, New Generation, Model A, Simplified, Standard, Terrace, Premium Apartment, DBSS      

In [10]:
# Combine 'flat_type' and 'flat_model' to give a better representation of the flat's category
data_hdb_last15['flat_category'] = data_hdb_last15['flat_type'] + " " + data_hdb_last15['flat_model']

In [11]:
# Change lease_commence_date to dt
data_hdb_last15['lease_commence_date'] = pd.to_datetime(data_hdb_last15['lease_commence_date'], format='%Y')

In [12]:
# Convert remaining lease from object to float

def convert_year_to_float(value):
    if isinstance(value, int):
        value = str(value)
    match = re.match(r'(\d+)(?: years (\d+) months)?', value)
    years = int(match.group(1))
    months = int(match.group(2)) if match.group(2) else 0
    return years + months / 12

data_hdb_last15['remaining_lease'] = data_hdb_last15['remaining_lease'].apply(convert_year_to_float)

In [13]:
# Source : https://hecksrealty.com/singapore-district-guide/
data_hdb_last15['postal_code'] = data_hdb_last15['full_address'].str[-6:]

data_hdb_last15['postal_code'] = data_hdb_last15['postal_code'].where(data_hdb_last15['postal_code'].str.isnumeric(), other='')

def determine_region(row):
    postal_sector = str(row['postal_code'])[:2]
    town = row['town']
    
    if postal_sector in ['53', '54', '55', '82', '56', '57', '72', '73', '77', '78', '75', '76', '79', '80']:
        return 'North'
    elif postal_sector in ['14', '15', '16', '09', '10']:
        return 'South'
    elif postal_sector in ['34', '36', '36', '37',  '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '81', '51', '52']:
        return 'East'
    elif postal_sector in ['11', '12', '13', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71']:
        return 'West'
    elif postal_sector in ['20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33']:
        return 'Central' 
    elif postal_sector in ['01', '02', '03', '04', '05', '06', '07', '08', '17', '18', '19']:
        return 'City'
    else:
        return 'Unknown'

data_hdb_last15['region'] = data_hdb_last15.apply(determine_region, axis=1)

def determine_region2(row):
    if row['region'] == 'Unknown':
        town = row['town']
        same_town_rows = data_hdb_last15[data_hdb_last15['town'] == town]
        if not same_town_rows.empty:
            return same_town_rows.iloc[0]['region']
    return row['region']

data_hdb_last15['region'] = data_hdb_last15.apply(determine_region2, axis=1)

print(data_hdb_last15)


              town flat_type flat_model  floor_area_sqm       street_name  \
1       ANG MO KIO    3 ROOM   Improved            61.0  ANG MO KIO AVE 4   
2       ANG MO KIO    3 ROOM   Improved            60.0  ANG MO KIO AVE 4   
3       ANG MO KIO    3 ROOM   Improved            69.0  ANG MO KIO AVE 4   
4       ANG MO KIO    2 ROOM   Improved            45.0  ANG MO KIO AVE 4   
5       ANG MO KIO    3 ROOM   Improved            61.0  ANG MO KIO AVE 4   
...            ...       ...        ...             ...               ...   
325966      YISHUN    2 ROOM    Model A            38.0      YISHUN AVE 4   
325967      YISHUN    4 ROOM    Model A            93.0      YISHUN AVE 4   
325968      YISHUN    3 ROOM    Model A            68.0      YISHUN AVE 4   
325969      YISHUN    5 ROOM   Improved           112.0      YISHUN AVE 4   
325970      YISHUN    5 ROOM   Improved           112.0      YISHUN AVE 4   

        resale_price      month lease_commence_date storey_range block  \
1

In [14]:
# Storey Range mapping for overlapping ranges
range_mapping = {
    '01 TO 03': '01 TO 05',
    '01 TO 05': '01 TO 05',
    '04 TO 06': '01 TO 05',
    '06 TO 10': '06 TO 10',
    '07 TO 09': '06 TO 10',
    '10 TO 12': '10 TO 15',
    '11 TO 15': '10 TO 15',
    '13 TO 15': '10 TO 15',
    '16 TO 18': '16 TO 20',
    '16 TO 20': '16 TO 20',
    '19 TO 21': '16 TO 20',
    '21 TO 25': '21 TO 25',
    '22 TO 24': '21 TO 25',
    '25 TO 27': '26 TO 30',
    '26 TO 30': '26 TO 30',
    '28 TO 30': '26 TO 30',
    '31 TO 33': '31 TO 35',
    '31 TO 35': '31 TO 35',
    '34 TO 36': '31 TO 35',
    '36 TO 40': '36 TO 40',
    '37 TO 39': '36 TO 40',
    '40 TO 42': '41 TO 50',
    '43 TO 45': '41 TO 50',
    '46 TO 48': '41 TO 50',
    '49 TO 51': '41 TO 50'
}

# Apply the mapping to the 'storey_range' column
data_hdb_last15['storey_range'] = data_hdb_last15['storey_range'].map(range_mapping)

# Check the unique values after mapping
print(data_hdb_last15['storey_range'].unique())

['01 TO 05' '06 TO 10' '10 TO 15' '21 TO 25' '16 TO 20' '26 TO 30'
 '31 TO 35' '36 TO 40' '41 TO 50']


In [15]:
# Check Updated HDB Dataset
data_hdb_last15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325970 entries, 1 to 325970
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   town                     325970 non-null  object        
 1   flat_type                325970 non-null  object        
 2   flat_model               325970 non-null  object        
 3   floor_area_sqm           325970 non-null  float64       
 4   street_name              325970 non-null  object        
 5   resale_price             325970 non-null  float64       
 6   month                    325970 non-null  datetime64[ns]
 7   lease_commence_date      325970 non-null  datetime64[ns]
 8   storey_range             325970 non-null  object        
 9   block                    325970 non-null  object        
 10  remaining_lease          325970 non-null  float64       
 11  address                  325970 non-null  object        
 12  full_address    

In [16]:
# Unique street_names
print(data_hdb_last15['street_name'].nunique())
data_hdb_last15['street_name'].unique()

566


array(['ANG MO KIO AVE 4', 'ANG MO KIO AVE 1', 'ANG MO KIO AVE 10',
       'ANG MO KIO AVE 3', 'BEDOK NTH RD', 'ANG MO KIO AVE 5',
       'ANG MO KIO AVE 6', 'ANG MO KIO AVE 8', 'ANG MO KIO AVE 9',
       'ANG MO KIO AVE 2', 'ANG MO KIO ST 21', 'ANG MO KIO ST 31',
       'ANG MO KIO ST 11', 'BEDOK RESERVOIR RD', 'BEDOK NTH ST 3',
       'BEDOK STH AVE 1', 'BEDOK STH RD', 'CHAI CHEE AVE',
       'NEW UPP CHANGI RD', 'CHAI CHEE DR', 'CHAI CHEE RD',
       'BEDOK NTH AVE 1', 'BEDOK NTH AVE 2', 'BEDOK NTH AVE 3',
       'BEDOK NTH AVE 4', 'BEDOK NTH ST 1', 'BEDOK NTH ST 2',
       'BEDOK NTH ST 4', 'BEDOK STH AVE 2', 'BEDOK STH AVE 3',
       'CHAI CHEE ST', 'LENGKONG TIGA', 'JLN TENAGA', 'SIN MING RD',
       'SHUNFU RD', 'BISHAN ST 11', 'BISHAN ST 12', 'BISHAN ST 13',
       'SIN MING AVE', 'BISHAN ST 22', 'BISHAN ST 23', 'BISHAN ST 24',
       'BRIGHT HILL DR', 'BT BATOK ST 51', 'BT BATOK ST 52',
       'BT BATOK ST 11', 'BT BATOK ST 21', 'BT BATOK ST 34',
       'BT BATOK WEST AVE 4', 

The dataset has 566 unique street names. By grouping the HDB units by street name to calculate the average coordinate of hdb units in the street, we can significantly cut down the time required to calculate the number of POIs within a certain radius of that average coordinate.

In [17]:
# getting average coordinates of each street name
street_avg_coords = data_hdb_last15.groupby('street_name').agg({'long': 'mean', 'lat': 'mean'}).reset_index()
street_avg_coords.columns = ['street_name', 'avg_long', 'avg_lat']

#ave_coord_df = pd.merge(data_hdb_last15, street_avg_coords, on='street_name', how='left')
#ave_coord_df
street_avg_coords

,street_name,avg_long,avg_lat
0,ADMIRALTY DR,103.815851,1.450018
1,ADMIRALTY LINK,103.816254,1.455540
2,AH HOOD RD,103.844667,1.327701
3,ALJUNIED AVE 2,103.886335,1.319203
4,ALJUNIED CRES,103.884427,1.320423
...,...,...,...
561,YUNG KUANG RD,103.721629,1.331872
562,YUNG LOH RD,103.722128,1.328313
563,YUNG PING RD,103.722079,1.329695
564,YUNG SHENG RD,103.721681,1.333492


### Import POI Dataset

In [18]:
# Importing Data
data_poi_raw = pd.read_csv('../dataset/points_of_interest.csv')
data_poi_raw

,Unnamed: 0,place_id,name,lat,lng,rating,user_ratings_total,price_level,formatted_address,global_code,...,train_station,natural_feature,subpremise,SUBZONE_NO,SUBZONE_N,SUBZONE_C,PLN_AREA_N,PLN_AREA_C,REGION_N,REGION_C
0,0,ChIJ01fgzLUe2jERxlhvImcbZ7g,Quayside Isle,1.247681,103.842072,4.3,568.0,NaN,"31 Ocean Way, Singapore 098375",6PH56RXR+3R,...,False,False,False,1.0,SENTOSA,SISZ01,SOUTHERN ISLANDS,SI,CENTRAL REGION,CR
1,1,ChIJ1S4qfY8Q2jERgb68gskzUbo,Sime Darby Centre,1.336644,103.783597,3.7,437.0,NaN,"896 Dunearn Rd, Singapore 589472",6PH58QPM+MC,...,False,False,False,2.0,SWISS CLUB,BTSZ02,BUKIT TIMAH,BT,CENTRAL REGION,CR
2,2,ChIJ1ZAIkrwZ2jERxtZGC1JnrHM,PoMo,1.300192,103.849220,3.8,1285.0,NaN,"1 Selegie Rd, Singapore 188306",6PH58R2X+3M,...,False,False,False,8.0,SELEGIE,RCSZ08,ROCHOR,RC,CENTRAL REGION,CR
3,3,ChIJ1ZYJOiAZ2jER1mvQqHstQII,LR boulangerie,1.293178,103.827194,4.3,12.0,NaN,"491 River Valley Rd, #01-02 valley point shopp...",6PH57RVG+7V,...,False,False,False,2.0,CHATSWORTH,TNSZ02,TANGLIN,TN,CENTRAL REGION,CR
4,4,ChIJ2Y1DYBI92jERlFUKKSznJrY,Tampines Hub,1.353108,103.940361,4.6,227.0,NaN,"1 Tampines Walk, Singapore 528523",6PH59W3R+64,...,False,False,False,3.0,TAMPINES WEST,TMSZ03,TAMPINES,TM,EAST REGION,ER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8667,8667,ChIJic0FbG4T2jERlg__g9KNSUU,Old Chang Kee (Head Office),1.450192,103.805305,4.3,25.0,1.0,"2 Woodlands Terrace, Singapore",6PH5FR24+34,...,False,False,False,9.0,SENOKO SOUTH,SBSZ09,SEMBAWANG,SB,NORTH REGION,NR
8668,8668,ChIJ7cjSMX0T2jERGYqGQog7A2E,Old Chang Kee @ Sun Plaza,1.448144,103.819983,3.4,10.0,1.0,"30 Sembawang Dr, #B1-44 Sun Plaza, Singapore",6PH5CRX9+7X,...,False,False,False,3.0,SEMBAWANG CENTRAL,SBSZ03,SEMBAWANG,SB,NORTH REGION,NR
8669,8669,ChIJxXwRE24T2jERDGyVjkkHTTs,Old Chang Kee Bldg,1.449830,103.805229,4.0,2.0,NaN,Singapore,6PH5CRX4+W3,...,False,False,False,9.0,SENOKO SOUTH,SBSZ09,SEMBAWANG,SB,NORTH REGION,NR
8670,8670,ChIJmQOMh1AT2jERs_1tteD7eTg,Old Chang Kee Coldstore,1.468055,103.812869,0.0,0.0,NaN,"20 Senoko Way, Singapore",6PH5FR97+64,...,False,False,False,8.0,SENOKO NORTH,SBSZ08,SEMBAWANG,SB,NORTH REGION,NR


In [19]:
# Understanding Data II
data_poi_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8672 entries, 0 to 8671
Columns: 125 entries, Unnamed: 0 to REGION_C
dtypes: bool(105), float64(6), int64(1), object(13)
memory usage: 2.2+ MB


In [20]:
# Understanding Data III
for i in range(len(data_poi_raw.columns)):
    print(data_poi_raw.columns[i])

Unnamed: 0
place_id
name
lat
lng
rating
user_ratings_total
price_level
formatted_address
global_code
compound_code
planning_area
brand
establishment
point_of_interest
store
food
health
restaurant
hospital
lodging
finance
cafe
convenience_store
clothing_store
atm
shopping_mall
grocery_or_supermarket
home_goods_store
school
bakery
beauty_salon
transit_station
place_of_worship
pharmacy
meal_takeaway
furniture_store
tourist_attraction
secondary_school
supermarket
doctor
shoe_store
dentist
jewelry_store
church
bank
primary_school
electronics_store
gym
spa
car_repair
pet_store
bus_station
university
park
general_contractor
subway_station
real_estate_agency
florist
hair_care
department_store
hardware_store
car_dealer
veterinary_care
travel_agency
bicycle_store
book_store
laundry
plumber
meal_delivery
lawyer
parking
mosque
physiotherapist
art_gallery
insurance_agency
bar
museum
storage
movie_theater
moving_company
liquor_store
gas_station
electrician
car_rental
locksmith
car_wash
post_office
e

In [21]:
# Retain POI types related to health, recreational areas and schools
columns_to_keep = ['name', 'rating', 'lat', 'lng', 'hospital', 'doctor', 'dentist', 'park', 'shopping_mall', 'amusement_park', 'zoo', 'library',
                   'school', 'primary_school', 'secondary_school', 'university']

data_poi_filtered = data_poi_raw[columns_to_keep]

# Create new columns for POI archetypes
data_poi_filtered['Healthcare'] = data_poi_filtered[['hospital', 'doctor', 'dentist']].any(axis=1)
data_poi_filtered['Recreational'] = data_poi_filtered[['park', 'shopping_mall', 'amusement_park', 'zoo', 'library']].any(axis=1)
data_poi_filtered['Education'] = data_poi_filtered[['school', 'primary_school', 'secondary_school', 'university']].any(axis=1)

# Drop rows where all selected POI types are False
# i.e. Dropping POIs that do not belong the Healthcare, Recreation or Education
data_poi_filtered = data_poi_filtered[data_poi_filtered.iloc[:, 4:].any(axis=1)]

data_poi_filtered = data_poi_filtered[['name', 'rating', 'lat', 'lng', 'Healthcare', 'Recreational', 'Education']]

data_poi_filtered

,name,rating,lat,lng,Healthcare,Recreational,Education
0,Quayside Isle,4.3,1.247681,103.842072,False,True,False
1,Sime Darby Centre,3.7,1.336644,103.783597,False,True,False
2,PoMo,3.8,1.300192,103.849220,False,True,False
4,Tampines Hub,4.6,1.353108,103.940361,False,True,False
5,City Plaza,3.8,1.314764,103.893408,False,True,False
...,...,...,...,...,...,...,...
7681,Orchid Garden & Koi Pond,4.7,1.353986,103.989008,False,True,False
7689,JCube,4.1,1.333310,103.740199,False,True,False
7770,Giant Panda Forest - River Safari,4.4,1.403751,103.792624,False,True,False
7939,Tiong Bahru Plaza,4.1,1.286560,103.827543,False,True,False


In [25]:
# Save Filtered POI Dataset
data_poi_filtered.to_csv('../dataset/data_poi_filtered.csv', index = False)

### Merge Datasets

In [22]:
# Function to calculate Haversine distance between two points
def haversine_distance(lng1, lat1, lng2, lat2):
    lng1, lat1, lng2, lat2 = map(radians, [lng1, lat1, lng2, lat2])
    distances = haversine_distances([[lat1, lng1], [lat2, lng2]])
    return distances[1, 0] * 6371.0  # multiply by Earth radius to get km

# Function to count POIs and return average rating of a certain type within a specified radius from given long and lat
def count_pois_within_radius(hdb_lng, hdb_lat, poi_data, poi_type, radius):
    count = 0
    total_rating = 0
    for index, poi in data_poi_filtered.iterrows():
        poi_lng = poi['lng']
        poi_lat = poi['lat']
        distance = haversine_distance(hdb_lng, hdb_lat, poi_lng, poi_lat)
        if distance <= radius and poi[poi_type] == True:
            count += 1
            total_rating += poi['rating']

    if(count == 0):
        average_rating = 0
    else:
        average_rating = total_rating / count
        
    return count, average_rating

In [23]:
# To test below function with a smaller subset of data
hdb_resale_data = street_avg_coords.sample(n = 10)
hdb_resale_data

,street_name,avg_long,avg_lat
6,ANCHORVALE CRES,103.889153,1.398368
66,BT BATOK EAST AVE 4,103.759098,1.349824
2,AH HOOD RD,103.844667,1.327701
410,SHUNFU RD,103.838177,1.350556
477,TESSENSOHN RD,103.856019,1.314459
563,YUNG PING RD,103.722079,1.329695
218,HOUGANG ST 51,103.889184,1.378941
13,ANG MO KIO AVE 10,103.856132,1.367981
150,CLEMENTI WEST ST 2,103.762321,1.304750
345,PASIR RIS ST 41,103.958236,1.372959


In [24]:
# Iterate through each unqiue Street Name as a proxy for the lat & long of the hdb unit
# Calculate the average rating for each POI archetype
for index, hdb_unit in street_avg_coords.iterrows():
    hdb_lng = hdb_unit['avg_long']
    hdb_lat = hdb_unit['avg_lat']

    healthcare_within_1km_count = 0
    recreational_within_1km_count = 0
    education_within_1km_count = 0
    healthcare_within_2km_count = 0
    recreational_within_2km_count = 0
    education_within_2km_count = 0
    healthcare_within_1km_average_rating = 0
    recreational_within_1km_average_rating = 0
    education_within_1km_average_rating = 0
    healthcare_within_2km_average_rating = 0
    recreational_within_2km_average_rating = 0
    education_within_2km_average_rating = 0
    
    # Count healthcare within 1km radius
    healthcare_within_1km_count, healthcare_within_1km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                                 'Healthcare', 1) 
    # Count recreational areas within 1km radius
    recreational_within_1km_count, recreational_within_1km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                                     'Recreational', 1)
    # Count educational instituitions within 1km radius
    education_within_1km_count, education_within_1km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                               'Education', 1)
    # Count healthcare within 2km radius
    healthcare_within_2km_count, healthcare_within_2km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                                 'Healthcare', 2) 
    # Count recreational areas within 1km radius
    recreational_within_2km_count, recreational_within_2km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                                     'Recreational', 2)
    # Count educational instituitions within 1km radius
    education_within_2km_count, education_within_2km_average_rating = count_pois_within_radius(hdb_lng, hdb_lat, data_poi_filtered,
                                                                                               'Education', 2)
    
    # Add the recreational_count and school_count columns to the original hdb dataset
    street_avg_coords.loc[index, 'healthcare_within_1km_count'] = healthcare_within_1km_count
    street_avg_coords.loc[index, 'healthcare_within_1km_average_rating'] = healthcare_within_1km_average_rating
    
    street_avg_coords.loc[index, 'recreational_within_1km_count'] = recreational_within_1km_count
    street_avg_coords.loc[index, 'recreational_within_1km_average_rating'] = recreational_within_1km_average_rating
    
    street_avg_coords.loc[index, 'education_within_1km_count'] = education_within_1km_count
    street_avg_coords.loc[index, 'education_within_1km_average_rating'] = education_within_1km_average_rating
    
    street_avg_coords.loc[index, 'healthcare_within_2km_count'] = healthcare_within_2km_count
    street_avg_coords.loc[index, 'healthcare_within_2km_average_rating'] = healthcare_within_2km_average_rating
    
    street_avg_coords.loc[index, 'recreational_within_2km_count'] = recreational_within_2km_count
    street_avg_coords.loc[index, 'recreational_within_2km_average_rating'] = recreational_within_2km_average_rating
    
    street_avg_coords.loc[index, 'education_within_2km_count'] = education_within_2km_count
    street_avg_coords.loc[index, 'education_within_2km_average_rating'] = education_within_2km_average_rating

street_avg_coords

,street_name,avg_long,avg_lat,healthcare_within_1km_count,healthcare_within_1km_average_rating,recreational_within_1km_count,recreational_within_1km_average_rating,education_within_1km_count,education_within_1km_average_rating,healthcare_within_2km_count,healthcare_within_2km_average_rating,recreational_within_2km_count,recreational_within_2km_average_rating,education_within_2km_count,education_within_2km_average_rating
0,ADMIRALTY DR,103.815851,1.450018,1.0,3.800000,3.0,3.733333,4.0,4.275000,2.0,3.900000,6.0,3.916667,8.0,4.250000
1,ADMIRALTY LINK,103.816254,1.455540,1.0,3.800000,3.0,3.733333,2.0,4.100000,1.0,3.800000,4.0,3.800000,5.0,4.140000
2,AH HOOD RD,103.844667,1.327701,48.0,3.147917,12.0,4.016667,4.0,4.425000,87.0,2.542529,16.0,3.962500,23.0,4.160870
3,ALJUNIED AVE 2,103.886335,1.319203,9.0,1.333333,6.0,4.016667,8.0,4.187500,23.0,1.708696,15.0,3.973333,15.0,4.173333
4,ALJUNIED CRES,103.884427,1.320423,7.0,1.714286,2.0,4.300000,6.0,4.100000,22.0,1.722727,15.0,3.973333,14.0,4.107143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,YUNG KUANG RD,103.721629,1.331872,1.0,3.300000,2.0,4.250000,2.0,4.300000,11.0,1.745455,4.0,4.375000,9.0,4.066667
562,YUNG LOH RD,103.722128,1.328313,1.0,3.300000,2.0,4.250000,2.0,4.300000,3.0,1.100000,4.0,4.350000,4.0,4.025000
563,YUNG PING RD,103.722079,1.329695,1.0,3.300000,2.0,4.250000,2.0,4.300000,7.0,1.828571,3.0,4.366667,6.0,4.033333
564,YUNG SHENG RD,103.721681,1.333492,1.0,3.300000,2.0,4.250000,3.0,4.033333,14.0,2.271429,5.0,4.300000,9.0,4.066667


In [26]:
# Merge counts to dataset
data_hdb_last15 = pd.merge(street_avg_coords, data_hdb_last15, on='street_name', how='left')

In [27]:
# Rearrange columns
cols = data_hdb_last15.columns.tolist()
data_hdb_last15 = data_hdb_last15[['town', 'flat_type', 'flat_model', 'floor_area_sqm', 'resale_price', 'month', 'lease_commence_date',
 'storey_range', 'block', 'remaining_lease', 'address', 'full_address', 'lat', 'long', 'nearest_mrt', 'nearest_distance_to_mrt',
 'flat_category', 'street_name', 'avg_long', 'avg_lat', 'healthcare_within_1km_count', 'healthcare_within_1km_average_rating', 
                                   'healthcare_within_2km_count','healthcare_within_2km_average_rating',
                                   'recreational_within_1km_count', 'recreational_within_1km_average_rating',
                                   'recreational_within_2km_count', 'recreational_within_2km_average_rating',
                                   'education_within_1km_count', 'education_within_1km_average_rating',
                                   'education_within_2km_count', 'education_within_2km_average_rating',
                                   'postal_code','region']]
data_hdb_last15

,town,flat_type,flat_model,floor_area_sqm,resale_price,month,lease_commence_date,storey_range,block,remaining_lease,...,recreational_within_1km_count,recreational_within_1km_average_rating,recreational_within_2km_count,recreational_within_2km_average_rating,education_within_1km_count,education_within_1km_average_rating,education_within_2km_count,education_within_2km_average_rating,postal_code,region
0,SEMBAWANG,5 ROOM,Premium Apartment,111.0,362000.0,2009-07-01,2001-01-01,01 TO 05,357A,91.000000,...,3.0,3.733333,6.0,3.916667,4.0,4.275,8.0,4.250,751357,North
1,SEMBAWANG,5 ROOM,Premium Apartment,110.0,370000.0,2009-08-01,2001-01-01,06 TO 10,357A,91.000000,...,3.0,3.733333,6.0,3.916667,4.0,4.275,8.0,4.250,751357,North
2,SEMBAWANG,5 ROOM,Premium Apartment,110.0,403000.0,2010-01-01,2001-01-01,16 TO 20,357A,90.000000,...,3.0,3.733333,6.0,3.916667,4.0,4.275,8.0,4.250,751357,North
3,SEMBAWANG,4 ROOM,Premium Apartment,95.0,350000.0,2010-07-01,2001-01-01,01 TO 05,357A,90.000000,...,3.0,3.733333,6.0,3.916667,4.0,4.275,8.0,4.250,751357,North
4,SEMBAWANG,4 ROOM,Premium Apartment,95.0,399000.0,2010-07-01,2001-01-01,10 TO 15,357A,90.000000,...,3.0,3.733333,6.0,3.916667,4.0,4.275,8.0,4.250,751357,North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325965,BUKIT MERAH,5 ROOM,Improved,114.0,921000.0,2022-10-01,1974-01-01,10 TO 15,87,50.750000,...,7.0,4.057143,78.0,4.052564,3.0,2.700,25.0,3.348,160087,South
325966,BUKIT MERAH,5 ROOM,Improved,117.0,930000.0,2022-10-01,1974-01-01,01 TO 05,87,50.750000,...,7.0,4.057143,78.0,4.052564,3.0,2.700,25.0,3.348,160087,South
325967,BUKIT MERAH,5 ROOM,Improved,117.0,978000.0,2022-12-01,1974-01-01,10 TO 15,87,50.666667,...,7.0,4.057143,78.0,4.052564,3.0,2.700,25.0,3.348,160087,South
325968,BUKIT MERAH,5 ROOM,Improved,114.0,950000.0,2022-12-01,1974-01-01,21 TO 25,87,50.583333,...,7.0,4.057143,78.0,4.052564,3.0,2.700,25.0,3.348,160087,South


In [28]:
# Check Merged Dataset
data_hdb_last15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325970 entries, 0 to 325969
Data columns (total 34 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   town                                    325970 non-null  object        
 1   flat_type                               325970 non-null  object        
 2   flat_model                              325970 non-null  object        
 3   floor_area_sqm                          325970 non-null  float64       
 4   resale_price                            325970 non-null  float64       
 5   month                                   325970 non-null  datetime64[ns]
 6   lease_commence_date                     325970 non-null  datetime64[ns]
 7   storey_range                            325970 non-null  object        
 8   block                                   325970 non-null  object        
 9   remaining_lease                      

### Saving Dataset

In [29]:
# Save Merged Dataset
data_hdb_last15.to_csv('../dataset/hdb_last15_merged.csv', index = False)